<a href="https://colab.research.google.com/github/vkrisvasan/llamaKV/blob/main/llamaindexYouTubeVideoCommentResponse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#This code is designed to fetch comments from a YouTube video,
#load the video's transcript, create a vector index from the transcript,
#and then use a large language model to respond to the top comments.
#create YOUTUBE_API_KEY in Google Developers Console >new project>Explore & Enable APIs.>
#navigate to YouTube Data API v3 under YouTube APIs>Enable the API>
#Create a credential>Note the API key

# Install required packages
!pip install llama-index llama-index-llms-groq groq llama-index-embeddings-huggingface llama-index-readers-youtube-transcript -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 861.9/861.9 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.

In [3]:
import os
from googleapiclient.discovery import build
from llama_index.llms.groq import Groq
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader
from llama_index.core import (VectorStoreIndex, StorageContext, load_index_from_storage,Settings)


In [4]:
# Import os and getpass for handling credentials
import os
import getpass
# Prompt for credentials if not found in environment variables
credential_names = ["GROQ_API_KEY","YOUTUBE_API_KEY"]
for credential in credential_names:
  if credential not in os.environ:
    os.environ[credential]=getpass.getpass("Provide your..." + credential)

Provide your...GROQ_API_KEY··········
Provide your...YOUTUBE_API_KEY··········


In [5]:
VIDEO_ID = "Kbk9BiPhm7o"  # Replace with your actual video ID
# Set up API key and video ID
YOUTUBE_API_KEY = os.environ['YOUTUBE_API_KEY']

# Step 1: Initialize the YouTube API client
youtube = build('youtube', 'v3', developerKey=YOUTUBE_API_KEY)

In [6]:
def get_comments(video_id, max_results=100):
    comments = []
    response = youtube.commentThreads().list(
        part="snippet",
        videoId=video_id,
        maxResults=max_results,
        textFormat="plainText"
    ).execute()

    for item in response.get('items', []):
        comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comments.append(comment)

    return comments

In [7]:
# Fetch comments from the video
comments = get_comments(VIDEO_ID)
print(f"Fetched {len(comments)} comments")

Fetched 100 comments


In [8]:
# Initialize the Groq Llama-8B model
llm = Groq(model="llama-3.1-8b-instant", api_key=os.environ["GROQ_API_KEY"])
print("LLM initialized")
Settings.llm = llm

# Initialize the embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Embedding model initialized")
Settings.embed_model = embed_model

#  Load YouTube Transcript
youtube_link = f"https://www.youtube.com/watch?v={VIDEO_ID}"
loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=[youtube_link])
print("Documents loaded")

# Create a Vector Index from the Transcript
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
print("Index created")



LLM initialized


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model initialized
Documents loaded
Index created


In [9]:
# Persist the Index to Storage
persist_dir = "./youtube_index_storage"
storage_context = StorageContext.from_defaults()

#index.storage_context = storage_context
index.storage_context.persist(persist_dir=persist_dir)
print("Index persisted")

# Load the Index from Storage
storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context=storage_context)
print("Index loaded from storage")

# Create a Query Engine from the Index
query_engine = index.as_query_engine(llm=llm)
print("Query engine created")

Index persisted
Index loaded from storage
Query engine created


In [17]:

# Respond to the Selected Comments [exclude the 1st comment position as it is metadata about the video]
start_position = int(input("There are 100s of comments, so select only specific comments2, Enter the starting position of the comment: "))
end_position = int(input("Enter the ending position of the comments: "))

top_comments = comments[start_position:end_position]

for comment in top_comments:
    response = query_engine.query(comment)
    print(f"**YouTube Comment: {comment}\n")
    print(f"**Langauge Model Response: {response}\n")


There are 100s of comments, so select only specific comments2, Enter the starting position of the comment: 23
Enter the ending position of the comments: 29
**YouTube Comment: We should have technology, when having a conversations we can see what they are thinking b, if they are safe , everything you want to know so you cannot be betrayed hurt ECT

**Langauge Model Response: Having a direct line to someone's thoughts and feelings could be incredibly empowering, but it also raises significant concerns about privacy and trust. If we could see what someone is thinking, it could potentially eliminate misunderstandings and hurt feelings, but it could also lead to a loss of personal boundaries and autonomy.

Imagine being able to see the thoughts and intentions of others, but not being able to control how they are perceived or used. It could create a sense of vulnerability and exposure, making it difficult to form genuine connections with others.

Furthermore, if we could see everything someo